In [ ]:
import sys
from pathlib import Path
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from cogwheel import data, gw_utils, gw_plotting, utils
from dot_pe import inference


# Generate synthetic data for inference

In [ ]:
# create data
event_data_kwargs = {
    "detector_names": "HLV",
    "duration": 120.0,
    "asd_funcs": ["asd_H_O3", "asd_L_O3", "asd_V_O3"],
    "tgps": 0.0,
    "fmax": 1600.0,
}


event_data = data.EventData.gaussian_noise(
    eventname="example_eventdata", **event_data_kwargs, seed=20250311
)

mchirp = 75
q = 1 / 2

m1, m2 = gw_utils.mchirpeta_to_m1m2(mchirp, gw_utils.q_to_eta(q))
injection_par_dic = dict(
    m1=m1,
    m2=m2,
    ra=0.5,
    dec=0.5,
    iota=np.pi * 1 / 3,
    psi=1.0,
    phi_ref=12.0,
    s1z=0.6,
    s2z=0.6,
    s1x_n=0.1,
    s1y_n=0.2,
    s2x_n=0.3,
    s2y_n=-0.2,
    l1=0.0,
    l2=0.0,
    tgps=0.0,
    f_ref=50.0,
    d_luminosity=5e3,
    t_geocenter=0.0,
)

event_data.inject_signal(injection_par_dic, "IMRPhenomXODE")

print(event_data.injection["d_h"] - event_data.injection["h_h"] / 2)
print(sum(event_data.injection["d_h"] - event_data.injection["h_h"] / 2))

Alternatively, load data form file

`event_data = data.EventData.from_npz(filename="../example_eventdata.npz")`

Set bank folder, in the relevant mass range.

See how to create mass ranges in create_sample_bank.ipynb


In [ ]:
bank_folder = "test_bank"
bank_folder = Path(bank_folder)
event_dir = event_data.eventname

In [ ]:
rundir = inference.run(
    event_dir=event_dir,
    event=event_data,
    bank_folder=bank_folder,
    n_int=2**16,
    n_ext=512,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**12,
    single_detector_blocksize=2**12,
    seed=42,
    size_limit=10**6,
    draw_subset=False,
    n_draws=None,
)

In [ ]:
rundirs = sorted(
    Path("example_eventdata").glob("run_*"),
    key=lambda x: int(x.name.split("_")[-1]),
)

rundir = rundirs[-1]
summary_results = utils.read_json(rundirs[-1] / "summary_results.json")

for k, v in summary_results.items():
    print(k, v)
    

In [ ]:
samples = pd.read_feather(rundirs[-1] / "samples.feather")
params = ["mchirp", "lnq", "chieff", "cumchidiff", "costheta_jn", "lnl"]
gw_plotting.CornerPlot(samples, params=params, smooth=1).plot()

In [ ]:
# Optional: Load existing results and run post-processing only
# can be used for debugging

event_data = data.EventData.from_npz(filename=rundir / "example_eventdata.npz")
bank_folder = Path("test_bank")
posterior = utils.read_json(rundir / "Posterior.json")
pr = posterior.prior
# Load the intrinsic indices that were used
intrinsic_data = np.load(rundir / "intrinsic_samples.npz")
inds = intrinsic_data["inds"]

# Run post-processing only
samples = inference.postprocess(
    event_data=event_data,
    rundir=rundir,
    bank_folder=bank_folder,
    n_int=2**16,
    inds=inds,
    n_ext=512,
    n_phi=32,
    pr=pr,
    draw_subset=False,
    n_draws=None,
)

# Save the new samples
samples.to_feather(rundir / "samples_postprocess_only.feather")
print(
    f"Post-processing completed. Samples saved to {rundir / 'samples_postprocess_only.feather'}"
)

# Compare with original results
original_samples = pd.read_feather(rundir / "samples.feather")
print(f"Original samples shape: {original_samples.shape}")
print(f"New samples shape: {samples.shape}")

In [ ]:
# Load existing results and run post-processing only
rundir = Path("example_eventdata/run_3")
event_data = data.EventData.from_npz(filename=rundir / "example_eventdata.npz")
bank_folder = Path("test_bank")
posterior = utils.read_json(rundir / "Posterior.json")
pr = posterior.prior

# Load the intrinsic indices that were used
intrinsic_data = np.load(rundir / "intrinsic_samples.npz")
inds = intrinsic_data["inds"]

# Run post-processing only
samples = inference.postprocess(
    event_data=event_data,
    rundir=rundir,
    bank_folder=bank_folder,
    n_int=2**16,
    inds=inds,
    n_ext=512,
    n_phi=32,
    pr=pr,
    draw_subset=False,
    n_draws=None,
)

# Load original samples for comparison
original_samples = pd.read_feather(rundir / "samples.feather")

print(f"Original shape: {original_samples.shape}")
print(f"New shape: {samples.shape}")
print(f"Columns match: {list(original_samples.columns) == list(samples.columns)}")

# Columns that change due to random luminosity distance sampling
random_columns = ['d_luminosity', 'd_h_1Mpc', 'h_h_1Mpc', 'lnl']

# Check for differences in specific columns (excluding random ones)
if not original_samples.equals(samples):
    print("\nDifferences found!")
    
    # Check which columns differ (excluding random columns)
    for col in original_samples.columns:
        if col not in random_columns and col in samples.columns:
            if not original_samples[col].equals(samples[col]):
                print(f"Column '{col}' differs")
                print(f"  Original first 5: {original_samples[col].head().values}")
                print(f"  New first 5: {samples[col].head().values}")
                print(f"  Max difference: {np.max(np.abs(original_samples[col] - samples[col]))}")
        elif col not in samples.columns:
            print(f"Column '{col}' missing in new samples")
    
    # Check for extra columns in new samples
    for col in samples.columns:
        if col not in original_samples.columns:
            print(f"Extra column '{col}' in new samples")
else:
    print("Results are identical!")

# Save the new samples for inspection
samples.to_feather(rundir / "samples_postprocess_only.feather")
print(f"\nNew samples saved to {rundir / 'samples_postprocess_only.feather'}")